# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的船票票號( 'Ticket' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 取 int64, float64 兩種數值型欄位, 存於 num_features 中
# 取類別值 (object) 型欄位, 存於 object_features 中
num_features = []
object_features = []

for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
    elif dtype == 'object':
        object_features.append(feature)
        
print(f'{len(object_features)} Numeric Features : {object_features}\n')
print(f'{len(object_features)} Object Features : {object_features}\n')

# 填補缺值
df_num = df[num_features]
df_num = df_num.fillna(df.mean())
df_obj = df[object_features] 
df_obj = df_obj.fillna('None')
train_num = train_Y.shape[0]
df = pd.concat([df_num,df_obj], axis=1)
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked
0,3,22.0,1,0,7.2500,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,1,38.0,1,0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,3,26.0,0,0,7.9250,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,1,35.0,1,0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,3,35.0,0,0,8.0500,"Allen, Mr. William Henry",male,373450,None,S


In [3]:
# 'Ticket' 欄位使用標籤編碼，與其他數值型欄位一起預估生存機率
df_temp = df_num
col = 'Ticket'
df_temp[col] = LabelEncoder().fit_transform(df[col])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
df_temp.head()

shape : (891, 6)
score : 0.692589511193576


/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/

,Pclass,Age,SibSp,Parch,Fare,Ticket
0,3,22.0,1,0,7.2500,720
1,1,38.0,1,0,71.2833,816
2,3,26.0,0,0,7.9250,914
3,1,35.0,1,0,53.1000,65
4,3,35.0,0,0,8.0500,649


In [4]:
# 'Ticket' 欄位使用目標均值編碼，與其他數值型欄位一起預估生存機率
col = 'Ticket'
data = df_obj[col]
data = pd.concat([data[:train_num], train_Y], axis=1)
mean_df = data.groupby(col)['Survived'].mean().reset_index()
mean_df.columns = [col, f'{col}_mean']
data = pd.merge(data, mean_df, on=col, how='left')
data = data.drop([col, 'Survived'] , axis=1)
data = pd.concat([df_num[:train_num], data], axis=1)
estimator = LogisticRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')
data.head()

shape : (891, 7)
score : 0.9484397225747904
time : 0.027551889419555664 sec


/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/

,Pclass,Age,SibSp,Parch,Fare,Ticket,Ticket_mean
0,3,22.0,1,0,7.2500,720,0.0
1,1,38.0,1,0,71.2833,816,1.0
2,3,26.0,0,0,7.9250,914,1.0
3,1,35.0,1,0,53.1000,65,0.5
4,3,35.0,0,0,8.0500,649,0.0


In [5]:
# 'Ticket' 欄位使用特徵雜湊，與其他數值型欄位一起預估生存機率
df_temp = copy.deepcopy(df)
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Ticket_Hash'] = df['Ticket'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7935512505022542


/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/

,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash
0,3,22.0,1,0,7.2500,155,1,720,185,3,6
1,1,38.0,1,0,71.2833,286,0,816,106,0,0
2,3,26.0,0,0,7.9250,523,0,914,185,3,1
3,1,35.0,1,0,53.1000,422,0,65,70,3,8
4,3,35.0,0,0,8.0500,22,1,649,185,3,5


# 作業2
* 承上題，三者比較效果何者最好?
-> 'Ticket' 欄位使用目標均值編碼，與其他數值型欄位一起預估生存機率

In [6]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df_obj.columns:
    df_temp[c] = LabelEncoder().fit_transform(df_obj[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/

0.780004837244799


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [7]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df_obj.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df_obj = pd.merge(df_obj, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [8]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df_obj[c])
df_temp['Cabin_Count'] = df_obj['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7856230275549181


/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [9]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df_obj[c])
df_temp['Cabin_Hash'] = df_obj['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7822457511861711


/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,8
1,286,0,816,106,0,2
2,523,0,914,185,3,8
3,422,0,65,70,3,5
4,22,1,649,185,3,8


In [10]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df_obj[c])
df_temp['Cabin_Hash'] = df_obj['Cabin'].map(lambda x:hash(x) % 10)
df_temp['Cabin_Count'] = df_obj['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7822269199765797


/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alfred/anaconda3/envs/ML100-Days/lib/python3.7/site-packages/sklearn/

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Count
0,155,1,720,185,3,8,1014
1,286,0,816,106,0,2,2
2,523,0,914,185,3,8,1014
3,422,0,65,70,3,5,2
4,22,1,649,185,3,8,1014
